# **WORD2VEC & PMI**

Summary:


1.   Import *narco_schedule*
2.   Import Word2Vec and Train Models
3.   Most Similar Terms




---

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
sys.path.append('/content/drive/My Drive/Università/inforet_prj/')

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()
import pandas as pd
import pickle
import spacy
import string
import seaborn as sns; sns.set()
from scipy.stats import norm
import matplotlib.pyplot as plt
import math

In [3]:
from nltk import ngrams
from gensim.models import Word2Vec
from gensim.models.keyedvectors import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import string

## **1. Import narco_schedule**

In [ ]:
with open("/content/drive/MyDrive/Università/inforet_prj/narco_schedule.pkl", "rb") as f:
  narco_schedule = pickle.load(f)

In [ ]:
narco_schedule = narco_schedule.loc[(narco_schedule.schedule != "0") ] #& (narco_schedule.schedule != "narco_3")
narco_schedule

,schedule,lemmatized
1,narco_1,issue consolidated appeal constitutionality en...
2,narco_2,indict count murder aggravate criminal sexual ...
3,narco_3,appeal jury verdict find sexually violent pers...
4,narco_4,charge information count bribery count possess...


In [ ]:
narco_schedule_1 = narco_schedule.loc[(narco_schedule.schedule == "narco_1") ] 
narco_schedule_1

,schedule,lemmatized
1,narco_1,appeal order dismiss untimely postconviction p...


In [ ]:
narco_schedule_2 = narco_schedule.loc[(narco_schedule.schedule == "narco_2") ] 
narco_schedule_2

,schedule,lemmatized
2,narco_2,indict count murder aggravate criminal sexual ...


## **2. Import Word2Vec and Train Models**

In [ ]:
!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2021-09-29 17:33:50--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.164.168
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.164.168|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘/root/input/GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  34.4MB/s    in 51s     

2021-09-29 17:34:42 (30.6 MB/s) - ‘/root/input/GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [ ]:
EMBEDDING_FILE = '/root/input/GoogleNews-vectors-negative300.bin.gz'
word_vectors = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

In [ ]:
n = 5
sequences_1 = {}
for sched, x in tqdm(zip(narco_schedule_1.schedule, narco_schedule_1.lemmatized), total=narco_schedule_1.shape[0]):
    penta_gram = ngrams(x.split(), n)

    sequences_1[sched] = ([[y[0], y[1], y[2], y[3], y[4]] for y in penta_gram])

100%|██████████| 1/1 [00:01<00:00,  1.60s/it]


In [ ]:
sequences_2 = {}
for sched, x in tqdm(zip(narco_schedule_2.schedule, narco_schedule_2.lemmatized), total=narco_schedule_2.shape[0]):
    penta_gram = ngrams(x.split(), n)

    sequences_2[sched] = ([[y[0], y[1], y[2], y[3], y[4]] for y in penta_gram])

100%|██████████| 1/1 [00:06<00:00,  6.01s/it]


---

In [ ]:
models_1 = {}
for schedule, sequence in sequences_1.items():
    models_1[schedule] = Word2Vec(sequences_1[schedule], min_count=10)

In [ ]:
models_2 = {}
for schedule, sequence in sequences_2.items():
    models_2[schedule] = Word2Vec(sequences_2[schedule], min_count=10)

---

## **3. Most Similar Terms**

### *3.1 Positive Similarity*

In [ ]:
schedule_1 = ["cannabis",  "marijuana",  "lsd", "heroin",  "ecstasy", "peyote", "mescaline", "mda" ] # 'methaqualone', , "mdma"
schedule_2 = ["cocaine", "methamphetamine", "dilaudid", "meperidine", "demerol",  "dexedrine", "fentanyl", "ritalin", "methadone", "amphetamine", "phencyclidine", "pseudoephedrine", "ephedrine", "meth", "opium", "dilaudid", "preludin"] #"hydromorphone", oxycodone

In [33]:
for genre, model in models_1.items():
    for y in schedule_1:
        print(genre, y, [(x[0], round(x[1], 2)) for x in model.wv.most_similar(positive=y)][:7])

narco_1 cannabis [('sativa', 0.57), ('gram', 0.56), ('heroin', 0.55), ('contraband', 0.52), ('marijuana', 0.51), ('substance', 0.51), ('drug', 0.51)]
narco_1 marijuana [('junk', 0.57), ('reefer', 0.55), ('pot', 0.54), ('dope', 0.54), ('peyote', 0.54), ('whiskey', 0.53), ('underage', 0.52)]
narco_1 lsd [('vile', 0.69), ('jug', 0.68), ('barbituric', 0.65), ('indole', 0.65), ('homogenous', 0.65), ('receptacle', 0.64), ('diacetyl', 0.64)]
narco_1 heroin [('narcotic', 0.72), ('drug', 0.6), ('cannabis', 0.55), ('junk', 0.52), ('marijuana', 0.51), ('peyote', 0.5), ('capsule', 0.49)]
narco_1 ecstasy [('pill', 0.65), ('balloon', 0.57), ('pentazocine', 0.57), ('capsule', 0.56), ('sandwich', 0.56), ('wrapper', 0.56), ('tin', 0.56)]
narco_1 peyote [('ounce', 0.71), ('codein', 0.67), ('downer', 0.65), ('dope', 0.64), ('junk', 0.64), ('spoon', 0.63), ('button', 0.63)]
narco_1 mescaline [('ingestion', 0.69), ('morphine', 0.68), ('bile', 0.67), ('chocolate', 0.65), ('cough', 0.64), ('milligram', 0.64)

In [ ]:
for genre, model in models_2.items():
    for y in schedule_2:
        print(genre, y, [(x[0], round(x[1], 2)) for x in model.wv.most_similar(positive=y)][:7])

narco_2 cocaine [('narcotic', 0.71), ('drug', 0.7), ('prepackage', 0.58), ('amphetamine', 0.58), ('gram', 0.57), ('ounce', 0.55), ('powder', 0.55)]
narco_2 methamphetamine [('pseudoephedrine', 0.65), ('phencyclidine', 0.55), ('cocaine', 0.54), ('meth', 0.53), ('ephedrine', 0.52), ('precursor', 0.52), ('manufacture', 0.52)]
narco_2 dilaudid [('demerol', 0.83), ('sulfate', 0.78), ('sulphate', 0.77), ('morphine', 0.76), ('ritalin', 0.72), ('secobarbital', 0.7), ('paregoric', 0.68)]
narco_2 meperidine [('phenmetrazine', 0.68), ('ritalin', 0.65), ('milophene', 0.64), ('hydrochlorate', 0.62), ('assortment', 0.62), ('anorexant', 0.62), ('laced', 0.62)]
narco_2 demerol [('dilaudid', 0.83), ('vitamin', 0.72), ('syrinx', 0.66), ('cc', 0.64), ('relaxer', 0.64), ('secobarbital', 0.63), ('analgesic', 0.63)]
narco_2 dexedrine [('amphetamine', 0.63), ('sulphate', 0.6), ('tablet', 0.6), ('paregoric', 0.59), ('caffeine', 0.59), ('barbiturate', 0.58), ('secobarbital', 0.58)]
narco_2 fentanyl [('osteoart

In [ ]:
def most_similar_list_value(model_, word, n):
    lista = []
    for genre, model in model_.items():
        for x in model.wv.most_similar(positive=word)[:n]:
            lista.append([x[0],x[1]])
    return lista

In [ ]:
narco_1_cannabis = most_similar_list_value(models_1, 'cannabis', 10)
narco_1_marijuana = most_similar_list_value(models_1, 'marijuana', 10)
narco_1_lsd = most_similar_list_value(models_1, 'lsd', 10)
narco_1_heroin = most_similar_list_value(models_1, 'heroin', 10)
narco_1_ecstasy = most_similar_list_value(models_1, 'ecstasy', 10)
narco_1_peyote = most_similar_list_value(models_1, 'peyote', 10)
narco_1_mescaline = most_similar_list_value(models_1, 'mescaline', 10)

terms_narco_1 = pd.concat([pd.DataFrame(narco_1_cannabis, columns=['cannabis', 'value']), 
                           pd.DataFrame(narco_1_marijuana, columns=['marijuana', 'value']), 
                          pd.DataFrame(narco_1_lsd, columns=['lsd', 'value']), 
                          pd.DataFrame(narco_1_heroin, columns=['heroin', 'value']), 
                          pd.DataFrame(narco_1_ecstasy, columns=['ecstasy', 'value']), 
                          pd.DataFrame(narco_1_peyote, columns=['peyote', 'value']),
                          pd.DataFrame(narco_1_mescaline, columns=['mescaline', 'value'])],                      
                          axis=1)
terms_narco_1

,cannabis,value,marijuana,value,lsd,value,heroin,value,ecstasy,value,peyote,value,mescaline,value
0,sativa,0.578546,junk,0.631975,homogenous,0.722048,narcotic,0.716593,wrapper,0.617202,ounce,0.652994,chocolate,0.706514
1,substance,0.543835,underage,0.567376,vile,0.720066,drug,0.574281,balloon,0.616958,spoon,0.641321,dose,0.687080
2,heroin,0.533344,pot,0.556687,jug,0.708156,cannabis,0.533344,diacetyl,0.612156,button,0.640277,bile,0.664728
3,drug,0.520599,reefer,0.536710,indole,0.703663,peyote,0.514641,pill,0.610881,pharmaceutical,0.638322,morphine,0.651704
4,contraband,0.505180,dope,0.531199,homogeneous,0.700429,tinfoil,0.499215,baggy,0.607405,junk,0.627794,ingestion,0.651621
5,gram,0.500301,telegram,0.517318,diacetyl,0.688599,powder,0.498795,sandwich,0.596921,downer,0.619682,cough,0.647260
6,narcotic,0.493684,peyote,0.513370,barbituric,0.682098,dope,0.495525,pentazocine,0.584969,stash,0.617316,milligram,0.640967
7,marijuana,0.491293,sift,0.505902,freezer,0.674351,junk,0.483148,homogeneous,0.583011,price,0.616674,wine,0.633030
8,hashish,0.486318,owl,0.503789,metabolite,0.654253,capsule,0.470397,lsd,0.580535,stuff,0.614480,optimum,0.619187
9,manufacture,0.473566,drug,0.492534,derivative,0.645479,gram,0.463225,pear,0.576595,hypodermically,0.605316,opiate,0.616744


In [ ]:
narco_2_cocaine = most_similar_list_value(models_2, 'cocaine', 10)
narco_2_methamphetamine = most_similar_list_value(models_2, 'methamphetamine', 10)
narco_2_dilaudid = most_similar_list_value(models_2, 'dilaudid', 10)
narco_2_meperidine = most_similar_list_value(models_2, 'meperidine', 10)
narco_2_demerol = most_similar_list_value(models_2, 'demerol', 10)
narco_2_dexedrine = most_similar_list_value(models_2, 'dexedrine', 10)
narco_2_fentanyl = most_similar_list_value(models_2, 'fentanyl', 10)
narco_2_ritalin = most_similar_list_value(models_2, 'ritalin', 10)
narco_2_methadone = most_similar_list_value(models_2, 'methadone', 10)

terms_narco_2 = pd.concat([pd.DataFrame(narco_2_cocaine, columns=['cocaine', 'value']), 
                           pd.DataFrame(narco_2_methamphetamine, columns=['methamphetamine', 'value']), 
                          pd.DataFrame(narco_2_dilaudid, columns=['dilaudid', 'value']), 
                          pd.DataFrame(narco_2_meperidine, columns=['meperidine', 'value']), 
                          pd.DataFrame(narco_2_demerol, columns=['demerol', 'value']), 
                          pd.DataFrame(narco_2_dexedrine, columns=['dexedrine', 'value']), 
                          pd.DataFrame(narco_2_fentanyl, columns=['fentanyl', 'value']), 
                          pd.DataFrame(narco_2_ritalin, columns=['ritalin', 'value']), 
                          pd.DataFrame(narco_2_methadone, columns=['methadone', 'value'])],                     
                          axis=1)
terms_narco_2

,cocaine,value,methamphetamine,value,dilaudid,value,meperidine,value,demerol,value,dexedrine,value,fentanyl,value,ritalin,value,methadone,value
0,drug,0.72,pseudoephedrine,0.69,morphine,0.82,commercially,0.74,dilaudid,0.80,sulphate,0.62,fi,0.66,sulfate,0.82,outpatient,0.63
1,narcotic,0.66,ephedrine,0.59,demerol,0.80,tablespoon,0.74,sulphate,0.73,sulfate,0.61,relaxer,0.62,preludin,0.77,detoxification,0.63
2,sixteenth,0.55,manufacture,0.58,sulphate,0.79,tetracaine,0.73,blister,0.69,tetracaine,0.60,naval,0.62,sulphate,0.75,medically,0.58
3,powder,0.54,meth,0.56,sulfate,0.77,schizotypal,0.72,relaxant,0.69,amphetamine,0.59,oversedation,0.62,ulcer,0.74,program,0.57
4,kilogram,0.53,vick,0.54,preludin,0.73,matchhead,0.72,relaxer,0.67,dose,0.59,quieter,0.61,antianxiety,0.73,clinic,0.56
5,methamphetamine,0.53,simulation,0.53,opium,0.72,misfunction,0.72,suppressant,0.66,ritalin,0.59,speck,0.60,schizotypal,0.72,inpatient,0.56
6,phencyclidine,0.52,cocaine,0.53,ritalin,0.70,heredity,0.72,preludin,0.66,hydrochlorate,0.59,payout,0.60,ental,0.72,screening,0.56
7,gram,0.51,phencyclidine,0.52,prescription,0.68,benzoyl,0.72,intravenous,0.66,tablet,0.57,suppressant,0.60,sedation,0.72,therapist,0.56
8,ounce,0.51,narcotic,0.50,tablet,0.68,whitish,0.71,refill,0.65,preludin,0.57,ulcer,0.59,benzoyl,0.71,antidepressant,0.56
9,kilo,0.51,precursor,0.50,hydrochlorate,0.66,baster,0.71,cc,0.64,capsule,0.56,consign,0.59,meperidine,0.71,enroll,0.55


## **TIME for SCHEDULE 1**

In [5]:
with open("/content/drive/MyDrive/Università/inforet_prj/narco_data.pkl", "rb") as f:
  narco_data = pickle.load(f)

In [8]:
schedule_1 = ["cannabis",  "marijuana",  "lsd", "heroin", 'methaqualone', "ecstasy", "peyote", "mescaline", "mda", "mdma"] #https://www.dea.gov/drug-information/drug-scheduling
schedule_2 = ["cocaine", "methamphetamine", "hydromorphone", "dilaudid", "meperidine", "demerol", "oxycodone", "dexedrine", "fentanyl", "ritalin", "methadone", "amphetamine", "phencyclidine", "pseudoephedrine", "ephedrine", "meth", "opium", "dilaudid", "preludin"]
schedule_3 = ["ketamine", "anabolic" , "steroids",  "testosterone", "ketamine"]
schedule_4 = ["modafinil", "provigil", "adderall", "methylphenidate", "memantine", "axura", "soma", "xanax", "darvon", "darvocet", "valium", "ativan", "talwin", "ambien", "tramadol",  "ethclorvynol"]
schedule_5 = ["phenylpropanolamine", "lomotil", "motofen", "lyrica", "parepectolin", "tetracaine"]

In [92]:
narco_data_1 = narco_data[(narco_data['lemmatized'].str.contains("|".join(schedule_1))) & ~(narco_data['lemmatized'].str.contains("|".join(schedule_3))) & ~(narco_data['lemmatized'].str.contains("|".join(schedule_2))) & ~(narco_data['lemmatized'].str.contains("|".join(schedule_4))) & ~(narco_data['lemmatized'].str.contains("|".join(schedule_5)))] 
narco_data_2 = narco_data[(narco_data['lemmatized'].str.contains("|".join(schedule_2))) & ~(narco_data['lemmatized'].str.contains("|".join(schedule_3))) & ~(narco_data['lemmatized'].str.contains("|".join(schedule_1))) & ~(narco_data['lemmatized'].str.contains("|".join(schedule_4))) & ~(narco_data['lemmatized'].str.contains("|".join(schedule_5)))] 

In [93]:
narco_1_time = narco_data_1.drop(['type','text','author', 'id','spacy_nlp'], axis = 1)
narco_2_time = narco_data_2.drop(['type','text','author', 'id','spacy_nlp'], axis = 1)

---

In [12]:
narco_1_time.loc[(narco_1_time["year"] >= 1950) & (narco_1_time["year"] <= 1959), 'year'] = 50 # 19
narco_1_time.loc[(narco_1_time["year"] >= 1960) & (narco_1_time["year"] <= 1969), 'year'] = 60 # 170
narco_1_time.loc[(narco_1_time["year"] >= 1970) & (narco_1_time["year"] <= 1979), 'year'] = 70 # 476
narco_1_time.loc[(narco_1_time["year"] >= 1980) & (narco_1_time["year"] <= 1989), 'year'] = 80 # 355
narco_1_time.loc[(narco_1_time["year"] >= 1990) & (narco_1_time["year"] <= 1999), 'year'] = 90 # 226
narco_1_time.loc[(narco_1_time["year"] >= 2000) & (narco_1_time["year"] <= 2009), 'year'] = 10 # 205

In [13]:
narco_1_time.loc[narco_1_time["year"] == 50, 'year'] = '50s' 
narco_1_time.loc[narco_1_time["year"] == 60, 'year'] = '60s' 
narco_1_time.loc[narco_1_time["year"] == 70, 'year'] = '70s' 
narco_1_time.loc[narco_1_time["year"] == 80, 'year'] = '80s' 
narco_1_time.loc[narco_1_time["year"] == 90, 'year'] = '90s'
narco_1_time.loc[narco_1_time["year"] == 10, 'year'] = '00s'

In [19]:
narco_1_time.loc[narco_1_time.year == '90s'].shape

(226, 2)

In [20]:
narco_1_time.drop(narco_1_time.loc[(narco_1_time["year"] != '50s' ) & (narco_1_time["year"] != '60s') & (narco_1_time["year"] != '70s') & (narco_1_time["year"] != '80s') & (narco_1_time["year"] != '90s') & (narco_1_time["year"] != '00s')].index, inplace=True)

In [21]:
narco_1_time = narco_1_time.groupby('year', as_index=False).agg({'lemmatized' : ' '.join})  

In [22]:
narco_1_50s = narco_1_time.loc[narco_1_time['year']=="50s"] # 2507  
narco_1_60s = narco_1_time.loc[narco_1_time['year']=="60s"] 
narco_1_70s = narco_1_time.loc[narco_1_time['year']=="70s"]
narco_1_80s = narco_1_time.loc[narco_1_time['year']=="80s"]
narco_1_90s = narco_1_time.loc[narco_1_time['year']=="90s"]
narco_1_00s = narco_1_time.loc[narco_1_time['year']=="00s"]

In [23]:
with open("/content/drive/MyDrive/Università/inforet_prj/narco_1_time.pkl", "wb") as f:
    pickle.dump(narco_1_time, f)

---

In [ ]:
n = 5
sequences_narco_1_50s = {}
for sched, x in tqdm(zip(narco_1_50s.year, narco_1_50s.lemmatized), total=narco_1_50s.shape[0]):
    penta_gram = ngrams(x.split(), n)

    sequences_narco_1_50s[sched] = ([[y[0], y[1], y[2], y[3], y[4]] for y in penta_gram])

In [ ]:
sequences_narco_1_60s = {}
for sched, x in tqdm(zip(narco_1_60s.year, narco_1_60s.lemmatized), total=narco_1_60s.shape[0]):
    penta_gram = ngrams(x.split(), n)

    sequences_narco_1_60s[sched] = ([[y[0], y[1], y[2], y[3], y[4]] for y in penta_gram])

In [ ]:
sequences_narco_1_70s = {}
for sched, x in tqdm(zip(narco_1_70s.year, narco_1_70s.lemmatized), total=narco_1_70s.shape[0]):
    penta_gram = ngrams(x.split(), n)

    sequences_narco_1_70s[sched] = ([[y[0], y[1], y[2], y[3], y[4]] for y in penta_gram])

In [ ]:
sequences_narco_1_80s = {}
for sched, x in tqdm(zip(narco_1_80s.year, narco_1_80s.lemmatized), total=narco_1_80s.shape[0]):
    penta_gram = ngrams(x.split(), n)

    sequences_narco_1_80s[sched] = ([[y[0], y[1], y[2], y[3], y[4]] for y in penta_gram])

In [ ]:
sequences_narco_1_90s = {}
for sched, x in tqdm(zip(narco_1_90s.year, narco_1_90s.lemmatized), total=narco_1_90s.shape[0]):
    penta_gram = ngrams(x.split(), n)

    sequences_narco_1_90s[sched] = ([[y[0], y[1], y[2], y[3], y[4]] for y in penta_gram])

In [ ]:
sequences_narco_1_00s = {}
for sched, x in tqdm(zip(narco_1_00s.year, narco_1_00s.lemmatized), total=narco_1_00s.shape[0]):
    penta_gram = ngrams(x.split(), n)

    sequences_narco_1_00s[sched] = ([[y[0], y[1], y[2], y[3], y[4]] for y in penta_gram])

---

In [26]:
models_narco_1_50s = {}
for schedule, sequence in sequences_narco_1_50s.items():
    models_narco_1_50s[schedule] = Word2Vec(sequences_narco_1_50s[schedule], min_count=10)

In [50]:
models_narco_1_60s = {}
for schedule, sequence in sequences_narco_1_60s.items():
    models_narco_1_60s[schedule] = Word2Vec(sequences_narco_1_60s[schedule], min_count=10)

In [57]:
models_narco_1_70s = {}
for schedule, sequence in sequences_narco_1_70s.items():
    models_narco_1_70s[schedule] = Word2Vec(sequences_narco_1_70s[schedule], min_count=10)

In [58]:
models_narco_1_80s = {}
for schedule, sequence in sequences_narco_1_80s.items():
    models_narco_1_80s[schedule] = Word2Vec(sequences_narco_1_80s[schedule], min_count=10)

In [59]:
models_narco_1_90s = {}
for schedule, sequence in sequences_narco_1_90s.items():
    models_narco_1_90s[schedule] = Word2Vec(sequences_narco_1_90s[schedule], min_count=10)

In [60]:
models_narco_1_00s = {}
for schedule, sequence in sequences_narco_1_00s.items():
    models_narco_1_00s[schedule] = Word2Vec(sequences_narco_1_00s[schedule], min_count=10)

---

## **TIME for SCHEDULE 2**

In [94]:
narco_2_time.loc[(narco_2_time["year"] >= 1950) & (narco_2_time["year"] <= 1959), 'year'] = 50 # 25
narco_2_time.loc[(narco_2_time["year"] >= 1960) & (narco_2_time["year"] <= 1969), 'year'] = 60 # 59
narco_2_time.loc[(narco_2_time["year"] >= 1970) & (narco_2_time["year"] <= 1979), 'year'] = 70 # 192
narco_2_time.loc[(narco_2_time["year"] >= 1980) & (narco_2_time["year"] <= 1989), 'year'] = 80 # 570
narco_2_time.loc[(narco_2_time["year"] >= 1990) & (narco_2_time["year"] <= 1999), 'year'] = 90 # 1068
narco_2_time.loc[(narco_2_time["year"] >= 2000) & (narco_2_time["year"] <= 2009), 'year'] = 10 # 801

In [95]:
narco_2_time.loc[narco_2_time["year"] == 50, 'year'] = '50s' 
narco_2_time.loc[narco_2_time["year"] == 60, 'year'] = '60s' 
narco_2_time.loc[narco_2_time["year"] == 70, 'year'] = '70s' 
narco_2_time.loc[narco_2_time["year"] == 80, 'year'] = '80s' 
narco_2_time.loc[narco_2_time["year"] == 90, 'year'] = '90s'
narco_2_time.loc[narco_2_time["year"] == 10, 'year'] = '00s'

In [101]:
narco_2_time.loc[narco_2_time.year == '00s'].shape

(801, 2)

In [102]:
narco_2_time.drop(narco_2_time.loc[(narco_2_time["year"] != '50s' ) & (narco_2_time["year"] != '60s') & (narco_2_time["year"] != '70s') & (narco_2_time["year"] != '80s') & (narco_2_time["year"] != '90s') & (narco_2_time["year"] != '00s')].index, inplace=True)

In [103]:
narco_2_time = narco_2_time.groupby('year', as_index=False).agg({'lemmatized' : ' '.join})  

In [105]:
narco_2_50s = narco_2_time.loc[narco_2_time['year']=="50s"] #  
narco_2_60s = narco_2_time.loc[narco_2_time['year']=="60s"] 
narco_2_70s = narco_2_time.loc[narco_2_time['year']=="70s"]
narco_2_80s = narco_2_time.loc[narco_2_time['year']=="80s"]
narco_2_90s = narco_2_time.loc[narco_2_time['year']=="90s"]
narco_2_00s = narco_2_time.loc[narco_2_time['year']=="00s"]

In [106]:
with open("/content/drive/MyDrive/Università/inforet_prj/narco_2_time.pkl", "wb") as f:
    pickle.dump(narco_2_time, f)

---

In [ ]:
n = 5
sequences_narco_2_50s = {}
for sched, x in tqdm(zip(narco_2_50s.year, narco_2_50s.lemmatized), total=narco_2_50s.shape[0]):
    penta_gram = ngrams(x.split(), n)

    sequences_narco_2_50s[sched] = ([[y[0], y[1], y[2], y[3], y[4]] for y in penta_gram])

In [ ]:
sequences_narco_2_60s = {}
for sched, x in tqdm(zip(narco_2_60s.year, narco_2_60s.lemmatized), total=narco_2_60s.shape[0]):
    penta_gram = ngrams(x.split(), n)

    sequences_narco_2_60s[sched] = ([[y[0], y[1], y[2], y[3], y[4]] for y in penta_gram])

In [ ]:
sequences_narco_2_70s = {}
for sched, x in tqdm(zip(narco_2_70s.year, narco_2_70s.lemmatized), total=narco_2_70s.shape[0]):
    penta_gram = ngrams(x.split(), n)

    sequences_narco_2_70s[sched] = ([[y[0], y[1], y[2], y[3], y[4]] for y in penta_gram])

In [ ]:
sequences_narco_2_80s = {}
for sched, x in tqdm(zip(narco_2_80s.year, narco_2_80s.lemmatized), total=narco_2_80s.shape[0]):
    penta_gram = ngrams(x.split(), n)

    sequences_narco_2_80s[sched] = ([[y[0], y[1], y[2], y[3], y[4]] for y in penta_gram])

In [111]:
sequences_narco_2_90s = {}
for sched, x in tqdm(zip(narco_2_90s.year, narco_2_90s.lemmatized), total=narco_2_90s.shape[0]):
    penta_gram = ngrams(x.split(), n)

    sequences_narco_2_90s[sched] = ([[y[0], y[1], y[2], y[3], y[4]] for y in penta_gram])

100%|██████████| 1/1 [00:06<00:00,  6.79s/it]


In [112]:
sequences_narco_2_00s = {}
for sched, x in tqdm(zip(narco_2_00s.year, narco_2_00s.lemmatized), total=narco_2_00s.shape[0]):
    penta_gram = ngrams(x.split(), n)

    sequences_narco_2_00s[sched] = ([[y[0], y[1], y[2], y[3], y[4]] for y in penta_gram])

100%|██████████| 1/1 [00:01<00:00,  1.77s/it]


---

In [113]:
models_narco_2_50s = {}
for schedule, sequence in sequences_narco_2_50s.items():
    models_narco_2_50s[schedule] = Word2Vec(sequences_narco_2_50s[schedule], min_count=10)

In [114]:
models_narco_2_60s = {}
for schedule, sequence in sequences_narco_2_60s.items():
    models_narco_2_60s[schedule] = Word2Vec(sequences_narco_2_60s[schedule], min_count=10)

In [115]:
models_narco_2_70s = {}
for schedule, sequence in sequences_narco_2_70s.items():
    models_narco_2_70s[schedule] = Word2Vec(sequences_narco_2_70s[schedule], min_count=10)

In [116]:
models_narco_2_80s = {}
for schedule, sequence in sequences_narco_2_80s.items():
    models_narco_2_80s[schedule] = Word2Vec(sequences_narco_2_80s[schedule], min_count=10)

In [117]:
models_narco_2_90s = {}
for schedule, sequence in sequences_narco_2_90s.items():
    models_narco_2_90s[schedule] = Word2Vec(sequences_narco_2_90s[schedule], min_count=10)

In [121]:
models_narco_2_00s = {}
for schedule, sequence in sequences_narco_2_00s.items():
    models_narco_2_00s[schedule] = Word2Vec(sequences_narco_2_00s[schedule], min_count=10)

---

In [122]:
list_models_time_schedule_1 = [models_narco_1_50s, models_narco_1_60s, models_narco_1_70s, models_narco_1_80s, models_narco_1_90s, models_narco_1_00s]
list_models_time_schedule_2 = [models_narco_2_50s, models_narco_2_60s, models_narco_2_70s, models_narco_2_80s, models_narco_2_90s, models_narco_2_00s]

In [135]:
def similarity_2_terms(word1, word2, list_models_schedule):
    print('(' , word1 , '-' , word2, ')')
    for x in list_models_schedule:
        try:
            for genre, model in x.items():
                print(genre, round(model.wv.similarity( word1, word2 ), 3) )
        except:
            pass

In [137]:
similarity_2_terms('drug', 'cocaine', list_models_time_schedule_2)

( drug - cocaine )
50s 0.765
60s 0.528
70s 0.422
80s 0.502
90s 0.704
00s 0.663


In [ ]:
for x in list_models_time:
    for genre, model in x.items():
            print(genre, model.wv.similarity('cannabis', 'drug'))
    #models_narco_1_50s.wv.similarity('cannabis', 'drug')

50s 0.9955919
60s 0.6859827
70s 0.2851925
80s 0.2523964
90s 0.41765204
00s 0.5667985


---

In [ ]:
drug = 'drug'
narcotic = 'narcotic'
crime = 'crime'
defendant = 'defendant'
addiction = 'addiction'

In [ ]:
for genre, model in models_1.items():
    print(genre, drug, [(x[0], round(x[1], 2)) for x in model.wv.most_similar(positive=drug)][:7])
    print(genre, narcotic, [(x[0], round(x[1], 2)) for x in model.wv.most_similar(positive=narcotic)][:7])
    print(genre, crime, [(x[0], round(x[1], 2)) for x in model.wv.most_similar(positive=crime)][:7])
    print(genre, defendant, [(x[0], round(x[1], 2)) for x in model.wv.most_similar(positive=defendant)][:7])
    print(genre, addiction, [(x[0], round(x[1], 2)) for x in model.wv.most_similar(positive=addiction)][:7])

narco_1 drug [('heroin', 0.63), ('narcotic', 0.6), ('junk', 0.57), ('dope', 0.55), ('peyote', 0.52), ('cannabis', 0.47), ('marijuana', 0.47)]
narco_1 narcotic [('heroin', 0.7), ('drug', 0.6), ('buyer', 0.5), ('junk', 0.49), ('dope', 0.49), ('illicit', 0.44), ('peyote', 0.44)]
narco_1 crime [('offense', 0.76), ('murder', 0.48), ('perjury', 0.46), ('lure', 0.44), ('robbery', 0.43), ('kidnapping', 0.43), ('burglary', 0.43)]
narco_1 defendant [('petitioner', 0.65), ('initially', 0.55), ('have', 0.47), ('far', 0.47), ('codefendant', 0.46), ('respondent', 0.46), ('previously', 0.43)]
narco_1 addiction [('addict', 0.66), ('habit', 0.59), ('ingest', 0.56), ('usage', 0.56), ('alcoholism', 0.52), ('user', 0.52), ('pusher', 0.52)]


In [ ]:
for genre, model in models_2.items():
    print(genre,  drug, [(x[0], round(x[1], 2)) for x in model.wv.most_similar(positive=drug)][:7])
    print(genre, narcotic, [(x[0], round(x[1], 2)) for x in model.wv.most_similar(positive=narcotic)][:7])
    print(genre, crime, [(x[0], round(x[1], 2)) for x in model.wv.most_similar(positive=crime)][:7])    
    print(genre, defendant, [(x[0], round(x[1], 2)) for x in model.wv.most_similar(positive=defendant)][:7])       
    print(genre, addiction, [(x[0], round(x[1], 2)) for x in model.wv.most_similar(positive=addiction)][:7])    

narco_2 drug [('narcotic', 0.82), ('cocaine', 0.72), ('buying', 0.54), ('profit', 0.5), ('dope', 0.49), ('occasional', 0.48), ('consummate', 0.47)]
narco_2 narcotic [('drug', 0.82), ('cocaine', 0.66), ('contraband', 0.51), ('methamphetamine', 0.5), ('substance', 0.49), ('buyer', 0.48), ('ongoe', 0.47)]
narco_2 crime [('offense', 0.7), ('peijury', 0.54), ('murder', 0.49), ('killing', 0.49), ('homicide', 0.48), ('oifense', 0.46), ('shooting', 0.45)]
narco_2 defendant [('initially', 0.55), ('codefendant', 0.49), ('petitioner', 0.46), ('finally', 0.46), ('have', 0.45), ('far', 0.44), ('respondent', 0.44)]
narco_2 addiction [('craving', 0.61), ('abuser', 0.61), ('drunkenness', 0.6), ('addict', 0.59), ('alcoholism', 0.58), ('psychotropic', 0.58), ('usage', 0.57)]


In [ ]:
lista = []
for genre, model in models_2.items():
    for x in model.wv.most_similar(positive='drug')[:7]:
        lista.append(x[0])

In [ ]:
def most_similar_list(model_, word, n):
    lista = []
    for genre, model in model_.items():
        for x in model.wv.most_similar(positive=word)[:n]:
            lista.append(x[0])
    return lista

In [ ]:
lista_drug_1 = most_similar_list(models_1, 'drug', 10)
lista_narcotic_1 = most_similar_list(models_1, 'narcotic', 10)
lista_crime_1 =  most_similar_list(models_1, 'crime', 10)
lista_defendant_1 =  most_similar_list(models_1, 'defendant', 10)
lista_addiction_1 =  most_similar_list(models_1, 'addiction', 10)
lista_drug_2 = most_similar_list(models_2, 'drug', 10)
lista_narcotic_2 = most_similar_list(models_2, 'narcotic', 10)
lista_crime_2 =  most_similar_list(models_2, 'crime', 10)
lista_defendant_2 =  most_similar_list(models_2, 'defendant', 10)
lista_addiction_2 =  most_similar_list(models_2, 'addiction', 10)

In [ ]:
similar_compare_drug = pd.concat([pd.DataFrame(lista_drug_1, columns=['drug_1']),  pd.DataFrame(lista_drug_2, columns=['drug_2'])], axis=1)
similar_compare_narcotic = pd.concat([pd.DataFrame(lista_narcotic_1, columns=['narcotic_1']),  pd.DataFrame(lista_narcotic_2, columns=['narcotic_2'])], axis=1)
similar_compare_crime = pd.concat([pd.DataFrame(lista_crime_1, columns=['crime_1']),  pd.DataFrame(lista_crime_2, columns=['crime_2'])], axis=1)
similar_compare_defendant = pd.concat([pd.DataFrame(lista_defendant_1, columns=['defendant_1']),  pd.DataFrame(lista_defendant_2, columns=['defendant_2'])], axis=1)
similar_compare_addiction = pd.concat([pd.DataFrame(lista_addiction_1, columns=['addiction_1']),  pd.DataFrame(lista_addiction_2, columns=['addiction_2'])], axis=1)

In [ ]:
pd.concat([similar_compare_drug, similar_compare_narcotic, similar_compare_crime, similar_compare_defendant, similar_compare_addiction ], axis=1)

,drug_1,drug_2,narcotic_1,narcotic_2,crime_1,crime_2,defendant_1,defendant_2,addiction_1,addiction_2
0,heroin,narcotic,heroin,drug,offense,offense,petitioner,initially,addict,craving
1,narcotic,cocaine,drug,cocaine,murder,peijury,initially,codefendant,habit,abuser
2,junk,buying,buyer,contraband,perjury,murder,have,petitioner,ingest,drunkenness
3,dope,profit,junk,methamphetamine,lure,killing,far,finally,usage,addict
4,peyote,dope,dope,substance,robbery,homicide,codefendant,have,alcoholism,alcoholism
5,cannabis,occasional,illicit,buyer,kidnapping,oifense,respondent,far,user,psychotropic
6,marijuana,consummate,peyote,ongoe,burglary,shooting,previously,respondent,pusher,usage
7,abuser,ongoe,amphetamine,middleman,shooting,obstruction,second,previously,overdose,ingestion
8,petty,middleman,cannabis,pseudoephedrine,transaction,robbery,additionally,lastly,withdrawal,intoxication
9,buyer,alcoholic,consummate,buying,actor,partly,trial,defense,dependency,rehabilitation


In [ ]:
lista_drug = most_similar_list(models_2, 'drug', 10)
lista_narcotic = most_similar_list(models_2, 'narcotic', 10)
most_similar_df = pd.concat([pd.DataFrame(lista_drug, columns=['drug']),  pd.DataFrame(lista_narcotic, columns=['narcotic'])], axis=1)
most_similar_df

,drug,narcotic
0,narcotic,drug
1,cocaine,cocaine
2,buying,contraband
3,profit,methamphetamine
4,dope,substance
5,occasional,buyer
6,consummate,ongoe
7,ongoe,middleman
8,middleman,pseudoephedrine
9,alcoholic,buying


In [ ]:
def most_similar_list_value(model_, word, n):
    lista = []
    for genre, model in model_.items():
        for x in model.wv.most_similar(positive=word)[:n]:
            lista.append([x[0], round(x[1], 2)])
    return lista

In [ ]:
lista_drug_v = most_similar_list_value(models_2, 'drug', 10)
lista_narcotic_v = most_similar_list_value(models_2, 'narcotic', 10)
most_similar_value_df = pd.concat([pd.DataFrame(lista_drug_v, columns=['drug', 'value']),  pd.DataFrame(lista_narcotic_v, columns=['narcotic', 'value'])], axis=1)
most_similar_value_df

,drug,value,narcotic,value
0,narcotic,0.82,drug,0.82
1,cocaine,0.72,cocaine,0.66
2,buying,0.54,contraband,0.51
3,profit,0.50,methamphetamine,0.50
4,dope,0.49,substance,0.49
5,occasional,0.48,buyer,0.48
6,consummate,0.47,ongoe,0.47
7,ongoe,0.46,middleman,0.45
8,middleman,0.46,pseudoephedrine,0.45
9,alcoholic,0.46,buying,0.45


---

### *3.2 Negative Similarity*

In [ ]:
for genre, model in models_1.items():
    print(genre, drug, [(x[0], round(x[1], 2)) for x in model.wv.most_similar(negative=['drug'])][:7])
    print(genre, narcotic, [(x[0], round(x[1], 2)) for x in model.wv.most_similar(negative=['narcotic'])][:7])
    print(genre, crime, [(x[0], round(x[1], 2)) for x in model.wv.most_similar(negative=['crime'])][:7])
    print(genre, defendant, [(x[0], round(x[1], 2)) for x in model.wv.most_similar(negative=['defendant'])][:7])
    print(genre, addiction, [(x[0], round(x[1], 2)) for x in model.wv.most_similar(negative=['addiction'])][:7])

In [ ]:
for genre, model in models_2.items():
    print(genre, drug, [(x[0], round(x[1], 2)) for x in model.wv.most_similar(negative=['word'])][:7])
    print(genre, narcotic, [(x[0], round(x[1], 2)) for x in model.wv.most_similar(negative=['narcotic'])][:7])
    print(genre, crime, [(x[0], round(x[1], 2)) for x in model.wv.most_similar(negative=['crime'])][:7])
    print(genre, defendant, [(x[0], round(x[1], 2)) for x in model.wv.most_similar(negative=['defendant'])][:7])
    print(genre, addiction, [(x[0], round(x[1], 2)) for x in model.wv.most_similar(negative=['addiction'])][:7])

---

## **4. Plotting Similarity**

In [ ]:
## SCHEDULE 1
words = ["cannabis",  "marijuana",  "heroin", "drug", "narcotic", "crime"]
similarities = np.zeros((len(words), len(words)), dtype=np.float_)
for idx1, word1 in enumerate(words):
    for idx2, word2 in enumerate(words):
        # note KeyError is possible if word doesn't exist
        sim = models_1['narco_1'].wv.similarity(word1, word2)
        similarities[idx1, idx2] = sim
        
df = pd.DataFrame.from_records(similarities, columns=words)
df.index = words

In [ ]:
f, ax=plt.subplots(1, 1, figsize=(14,8))
cmap = plt.cm.Blues
#mask = np.zeros_like(df)
#mask[np.triu_indices_from(mask)] = True
sns.heatmap(df, cmap=cmap, square=True,annot=True, ax=ax)
#_=plt.yticks(rotation=90)
plt.xlabel('Words')
#_=plt.xticks(rotation=45)
_=plt.title("Similarities between words SCHEDULE 1")

---

In [ ]:
## SCHEDULE 2
words = ["cocaine", "methamphetamine", "fentanyl", "drug", "narcotic", "crime"]
similarities = np.zeros((len(words), len(words)), dtype=np.float_)
for idx1, word1 in enumerate(words):
    for idx2, word2 in enumerate(words):
        # note KeyError is possible if word doesn't exist
        sim = models_2['narco_2'].wv.similarity(word1, word2)
        similarities[idx1, idx2] = sim
        
df = pd.DataFrame.from_records(similarities, columns=words)
df.index = words

In [ ]:
f, ax=plt.subplots(1, 1, figsize=(14,8))
cmap = plt.cm.Blues
axr = sns.heatmap(df, cmap=cmap, square=True ,annot=True, ax=ax, cbar=False)
plt.xlabel('Words')
_=plt.xticks(rotation=30)
_=plt.title("Similarities between words SCHEDULE 2")

In [ ]:
def display_pca_scatterplot(model, words=None, sample=0):
    if words == None:
        if sample > 0:
            words = np.random.choice(list(model.vocab.keys()), sample)
        else:
            words = [ word for word in model.vocab ]
        
    word_vectors = np.array([model[w] for w in words])

    twodim = PCA().fit_transform(word_vectors)[:,:2]
    
    plt.figure(figsize=(6,6))
    plt.scatter(twodim[:,0], twodim[:,1], edgecolors='k', c='r')
    for word, (x,y) in zip(words, twodim):
        plt.text(x+0.05, y+0.05, word)

In [ ]:
['relitigation','ar', 'dence','social','spine','chairman','doctorate','conformity','cessation','posture','depress','revive','survive','dignity','roll','sick', 'brake','uncharted','noncompliant', 'focused', 'lip', 'evaporate','numeral',  'airway', 'ventilate','physiological', 'ventilation', 'strive', 
'hat', 'vinyl', 'ponytail', 'sweatshirt', 'beard','shirt', 'rust']

In [ ]:
display_pca_scatterplot(models_2['narco_2'], 
                        ["cocaine", "methamphetamine", "fentanyl", "drug", "narcotic", "crime", 'relitigation','ar', 'dence','social','spine','chairman','doctorate','conformity','cessation','posture','depress','revive','survive','dignity','roll','sick', 'brake','uncharted','noncompliant', 'focused', 'lip', 'evaporate','numeral',  'airway', 'ventilate','physiological', 'ventilation', 'strive', 
'hat', 'vinyl', 'ponytail', 'sweatshirt', 'beard','shirt', 'rust'])

In [ ]:
for genre, model in models_1.items():
    print(genre, model.wv.most_similar(positive=['narcotic'], negative=['man'])[0][0])

In [ ]:
for genre, model in models_2.items():
    print(genre, model.wv.most_similar(positive=['narcotic'], negative=['man'])[0][0])

---

In [ ]:
for genre, model in models_1.items():
    print(genre, model.wv.most_similar(positive=['defendant', 'trial'], 
                                    negative=['man'])[0][0])

In [ ]:
for genre, model in models_1.items():
    print(genre, model.wv.most_similar(positive=['defendant', 'trial'], 
                                    negative=['man'])[0][0])

## **5. PMI**

In [ ]:
narco_1_pmi_nlp = pd.read_csv("/content/drive/MyDrive/Università/inforet_prj/narco_1_pmi_nlp.csv", sep="|" , names=["sentences_lemma"] )
narco_1_pmi = pd.read_csv("/content/drive/MyDrive/Università/inforet_prj/narco_1_pmi.csv", sep="|" )

In [ ]:
narco_1_pmi = pd.concat([narco_1_pmi, narco_1_pmi_nlp], axis=1)
narco_1_pmi = narco_1_pmi.drop(['type','text','author', 'sentences_joined'], axis = 1)

In [ ]:
narco_1_pmi =narco_1_pmi[:2]
narco_1_pmi.sentences_lemma[0]

In [ ]:
## https://shravan-kuchkula.github.io/nlp/document_similarity/#normalize-the-corpus-of-documents

In [ ]:
## PMI CO-OCCURRENCE
from collections import defaultdict
def co_occurrence(sentences, window_size):
    d = defaultdict(int)
    vocab = set()
    for text in sentences:
        # preprocessing (use tokenizer instead)
        text = text.lower().split()
        # iterate over sentences
        for i in range(len(text)):
            token = text[i]
            vocab.add(token)  # add to vocab
            next_token = text[i+1 : i+1+window_size]
            for t in next_token:
                key = tuple( sorted([t, token]) )
                d[key] += 12

    # formulate the dictionary into dataframe
    vocab = sorted(vocab) # sort vocab
    df = pd.DataFrame(data=np.zeros((len(vocab), len(vocab)), dtype=np.int16),
                      index=vocab,
                      columns=vocab)
    for key, value in d.items():
        df.at[key[0], key[1]] = value
        df.at[key[1], key[0]] = value
    return df

In [ ]:
co_matrix = co_occurrence(narco_1_pmi.sentences_lemma, 2)

In [ ]:
def pmi(df, positive=True):
    col_totals = df.sum(axis=0)
    total = col_totals.sum()
    row_totals = df.sum(axis=1)
    expected = np.outer(row_totals, col_totals) / total
    df = df / expected
    # Silence distracting warnings about log(0):
    with np.errstate(divide='ignore'):
        df = np.log(df)
    df[np.isinf(df)] = 0.0  # log(0) = 0
    if positive:
        df[df < 0] = 0.0
    return df

In [ ]:
co_matrix = pmi(co_matrix, positive=True)
co_matrix

In [ ]:
co_matrix.loc["sell","drug"]

In [ ]:
maxValue = np.amax(co_matrix['sell'])
maxValue

In [ ]:
minValue = np.amin(co_matrix['sell'])
minValue

In [ ]:
def nbest(self, score_fn, n):
    """Returns the top n ngrams when scored by the given function."""
    return [p for p,s in self.score_ngrams(score_fn)[:n]]

In [ ]:
def score_ngrams(self, score_fn):
    """Returns a sequence of (ngram, score) pairs ordered from highest to
    lowest score, as determined by the scoring function provided.
    """
    return sorted(self._score_ngrams(score_fn),
                  key=_itemgetter(1), reverse=True)

In [ ]:
text = [word_tokenize(x) for x in narco_schedule_1.lemmatized]
#text = ' '.join(text)
text = ','.join(str(v) for v in text)
text = text.replace(',', '')
text = text.replace("'", '')
text = text.replace('[', '')
text = text.replace(']', '')

In [ ]:
text

In [ ]:
import nltk
nltk.download('punkt')
from nltk.collocations import *
from nltk.tokenize import word_tokenize

#text = narco_schedule_1.lemmatized

trigram_measures = nltk.collocations.TrigramAssocMeasures()
finder = TrigramCollocationFinder.from_words(word_tokenize(text))

for i in finder.score_ngrams(trigram_measures.pmi)[:20]:
    print(i)

In [ ]:
narco_schedule = narco_schedule.loc[(narco_schedule.schedule != "0") ] #& (narco_schedule.schedule != "narco_3")
narco_schedule

---

## **6. Similarity between documents**

In [ ]:
narco_schedule_1 = narco_schedule.loc[narco_schedule['schedule']=="narco_1"] # 1642 
narco_schedule_2 = narco_schedule.loc[narco_schedule['schedule']=="narco_2"] # 2007 
narco_schedule_3 = narco_schedule.loc[narco_schedule['schedule']=="narco_3"] # 12
narco_schedule_4 = narco_schedule.loc[narco_schedule['schedule']=="narco_4"] # 2

In [ ]:
sched_ = [narco_schedule_1.lemmatized, narco_schedule_2.lemmatized, narco_schedule_3.lemmatized, narco_schedule_4.lemmatized]
num_sched_ = len(sched_)

In [ ]:
num_sched_

In [ ]:
sched_id_ = ['schedule_1', 'schedule_2', 'schedule_3','schedule_4']
sched_dict_ = dict(zip(sched_id_, sched_))

In [ ]:
ids_ = list(sched_dict_.keys())

In [ ]:
pairs_ = []
# create a list of tuples
for i, v in enumerate(ids_):
    for j in ids_[i+1:]:
        pairs_.append((ids_[i], j))

In [ ]:
pairs_

In [ ]:
feature_matrix_ = vectorizer.fit_transform(narco_schedule).astype(float)

In [ ]:
narco_schedule

In [ ]:
def compute_cosine_similarity(pair):

    # extract the indexes from the pair
    sc1, sc2 = pair

    # split on _ and get index
    sc1_index = int(sc1.split("_")[1])
    sc2_index = int(sc2.split("_")[1])

    # get the feature matrix of the document
    sc1_fm = feature_matrix_.toarray()[sc1_index]
    sc2_fm = feature_matrix_.toarray()[sc2_index]

    # compute cosine similarity manually
    manual_cosine_similarity = np.dot(sc1_fm, sc2_fm)

    return manual_cosine_similarity

In [ ]:
def compute_cosine_similarity(pair):

    # extract the indexes from the pair
    book1, book2 = pair

    # split on _ and get index
    book1_index = int(book1.split("_")[1])
    book2_index = int(book2.split("_")[1])

    # get the feature matrix of the document
    book1_fm = feature_matrix_.toarray()[book1_index]
    book2_fm = feature_matrix_.toarray()[book2_index]

    # compute cosine similarity manually
    manual_cosine_similarity = np.dot(book1_fm, book2_fm)

    return manual_cosine_similarity

In [ ]:
pairwise_cosine_similarity_ = [compute_cosine_similarity(pair) for pair in pairs_]

# create a dataframe
df_ = pd.DataFrame({'pair': pairs_, 'similarity': pairwise_cosine_similarity_})
display(df_.head())

In [ ]:
feature_matrix_.toarray()[2]

In [ ]:
b= [pairs[0],compute_cosine_similarity(pairs[0])]
b

In [ ]:
def doc_similarity_list(pairs):
    lista = []
    for pair in pairs:
        lista.append([pair, compute_cosine_similarity(pair)])

    return lista

In [ ]:
print(np.asarray(pairs).shape)

---

In [ ]:
with open("/content/drive/MyDrive/Università/inforet_prj/narco_1_time.pkl", "rb") as f:
  narco_1_time = pickle.load(f)

In [ ]:
narco_1_time

In [ ]:
with open("/content/drive/MyDrive/Università/inforet_prj/narco_data.pkl", "rb") as f:
  narco_data = pickle.load(f)

In [ ]:
schedule_1 = ["cannabis",  "marijuana",  "lsd", "heroin", 'methaqualone', "ecstasy", "peyote" ]
schedule_2 = ["cocaine", "methamphetamine", "hydromorphone", "dilaudid", "meperidine", "demerol", "oxycodone", "dexedrine", "fentanyl", "ritalin", "methadone"]
#schedule_3 = ["ketamine", "anabolic" , "steroids",  "testosterone"]
#schedule_4 = ["modafinil", "provigil", "adderall", "methylphenidate", "memantine", "axura"]

In [ ]:
narco_data_1 = narco_data[(narco_data['lemmatized'].str.contains("|".join(schedule_1))) & ~(narco_data['lemmatized'].str.contains("|".join(schedule_3))) & ~(narco_data['lemmatized'].str.contains("|".join(schedule_2))) & ~(narco_data['lemmatized'].str.contains("|".join(schedule_4)))] # 1741 
narco_1_time = narco_data_1.drop(['type','text','author', 'id','spacy_nlp'], axis = 1)

In [ ]:
narco_1_time.loc[(narco_1_time["year"] >= 1950) & (narco_1_time["year"] <= 1959), 'year'] = 50 # 33
narco_1_time.loc[(narco_1_time["year"] >= 1960) & (narco_1_time["year"] <= 1969), 'year'] = 60 # 222
narco_1_time.loc[(narco_1_time["year"] >= 1970) & (narco_1_time["year"] <= 1979), 'year'] = 70 # 560
narco_1_time.loc[(narco_1_time["year"] >= 1980) & (narco_1_time["year"] <= 1989), 'year'] = 80 # 414
narco_1_time.loc[(narco_1_time["year"] >= 1990) & (narco_1_time["year"] <= 1999), 'year'] = 90 # 251
narco_1_time.loc[(narco_1_time["year"] >= 2000) & (narco_1_time["year"] <= 2009), 'year'] = 10 # 229

In [ ]:
narco_1_time.loc[narco_1_time["year"] == 50, 'year'] = '50s' 
narco_1_time.loc[narco_1_time["year"] == 60, 'year'] = '60s' 
narco_1_time.loc[narco_1_time["year"] == 70, 'year'] = '70s' 
narco_1_time.loc[narco_1_time["year"] == 80, 'year'] = '80s' 
narco_1_time.loc[narco_1_time["year"] == 90, 'year'] = '90s'
narco_1_time.loc[narco_1_time["year"] == 10, 'year'] = '00s'

In [ ]:
narco_1_time.drop(narco_1_time.loc[(narco_1_time["year"] != '50s' ) & (narco_1_time["year"] != '60s') & (narco_1_time["year"] != '70s') & (narco_1_time["year"] != '80s') & (narco_1_time["year"] != '90s') & (narco_1_time["year"] != '00s')].index, inplace=True)
narco_1_time = narco_1_time.groupby('year', as_index=False).agg({'lemmatized' : ' '.join})  

In [ ]:
narco_1_50s = narco_1_time.loc[narco_1_time['year']=="50s"] # 2507  
narco_1_60s = narco_1_time.loc[narco_1_time['year']=="60s"] 
narco_1_70s = narco_1_time.loc[narco_1_time['year']=="70s"]
narco_1_80s = narco_1_time.loc[narco_1_time['year']=="80s"]
narco_1_90s = narco_1_time.loc[narco_1_time['year']=="90s"]
narco_1_00s = narco_1_time.loc[narco_1_time['year']=="00s"]

In [ ]:
sched = [narco_1_50s.lemmatized, narco_1_60s.lemmatized, narco_1_70s.lemmatized, narco_1_80s.lemmatized, narco_1_90s.lemmatized, narco_1_00s.lemmatized]
num_sched = len(sched)

In [ ]:
sched_id = ['1_50_s', '2_60_s', '3_70_s','4_80_s', '5_90_s', '0_00_s']
sched_dict = dict(zip(sched_id, sched))
ids = list(sched_dict.keys())

In [ ]:
pairs = []
# create a list of tuples
for i, v in enumerate(ids):
    for j in ids[i+1:]:
        pairs.append((ids[i], j))

In [ ]:
vectorizer = TfidfVectorizer()
feature_matrix = vectorizer.fit_transform(narco_1_time)

In [ ]:
def compute_cosine_similarity(pair):

    # extract the indexes from the pair
    book1, book2 = pair

    # split on _ and get index
    book1_index = int(book1.split("_")[0])
    book2_index = int(book2.split("_")[0])

    # get the feature matrix of the document
    book1_fm = feature_matrix.toarray()[book1_index]
    book2_fm = feature_matrix.toarray()[book2_index]

    # compute cosine similarity manually
    manual_cosine_similarity = np.dot(book1_fm, book2_fm)

    return manual_cosine_similarity

In [ ]:
feature_matrix.toarray()[3]

In [ ]:
pairwise_cosine_similarity = [compute_cosine_similarity(pair) for pair in pairs]

# create a dataframe
df = pd.DataFrame({'pair': pairs, 'similarity': pairwise_cosine_similarity})
display(df)


In [ ]:
b = [pairs[0],compute_cosine_similarity(pairs[0])]
b

In [ ]:
def doc_similarity_list(pairs):
    lista = []
    for pair in pairs:
        lista.append([pair, compute_cosine_similarity(pair)])

    return lista

In [ ]:
print(np.asarray(pairs).shape)

In [ ]:
doc_similarity_list(pairs[:1])